In [1]:
import numpy as np
from selection.algorithms.lasso import instance
from selection.algorithms.forward_step import forward_stepwise
np.random.seed(0)
from compute_pvalues import compute_pvalues

## 10th step, snr=5

In [2]:
X, y, beta, active, sigma = instance(n=100, p=40, snr=5, rho=0.3)
n, p = X.shape
FS = forward_stepwise(X, y, covariance=sigma**2 * np.identity(n))
for _ in range(10):
    FS.next()

FS.X.shape, FS.Y.shape

((100, 40), (100,))

In [3]:
con = FS.constraints()
con.linear_part.shape

(700, 100)

In [4]:
P = np.dot(FS.X[:,FS.variables[:-1]], np.linalg.pinv(FS.X[:,FS.variables[:-1]]))
R = np.identity(n) - P

In [5]:
def null_sample(sigma, R, PY):
    return sigma * np.dot(R, np.random.standard_normal(n)) + PY

In [6]:
def acceptance(con, sigma, R, PY):
    idx = 0
    while True:
        Z = null_sample(sigma, R, PY)
        if con(Z):
            break
        idx += 1
         
    return idx
PY = np.dot(P, FS.Y)
acceptance(con, sigma, R, PY)

391

In [7]:
[acceptance(con, sigma, R, PY) for _ in range(10)]

[94, 172, 43, 32, 145, 65, 380, 40, 52, 226]

In [8]:
FS.variables

[3, 6, 5, 2, 7, 1, 4, 30, 11, 13]

Let's make a function to see how many accept reject steps it takes.

In [9]:
del([X,y,FS,R,P,con,beta,active,sigma])

In [10]:
def howlong(n=100, p=40, snr=5, rho=0.3, maxstep=10):
    X, y, beta, active, sigma = instance(n=n, p=p, snr=snr, rho=rho)
    n, p = X.shape
    FS = forward_stepwise(X, y, covariance=sigma**2 * np.identity(n))
    for _ in range(maxstep):
        FS.next()

    con = FS.constraints()
    P = np.dot(FS.X[:,FS.variables[:-1]], np.linalg.pinv(FS.X[:,FS.variables[:-1]]))
    R = np.identity(n) - P
    PY = np.dot(P, FS.Y)
    
    return acceptance(con, sigma, R, PY)

## snr=4

In [11]:
[howlong(snr=4) for _ in range(10)]

[37, 187, 4267, 801, 90, 188, 63874, 107, 82, 1812]

## p=60, snr=5

In [12]:
[howlong(snr=5, p=60) for _ in range(10)]

[1799, 1039, 142, 400, 118, 2944, 359, 250, 357, 727]

## p=60, snr=5, 12th step

In [13]:
[howlong(snr=5, p=60, maxstep=12) for _ in range(10)]

[56, 24157, 594, 2179, 2304, 403, 8986, 5950, 181, 242]

## p=80, snr=5

In [14]:
[howlong(snr=5, p=80) for _ in range(10)]

[43, 375, 427, 100, 233, 626, 682, 494, 517, 19247]

## p=80, snr=4

In [15]:
[howlong(snr=4, p=80) for _ in range(10)]

[1443, 916, 68, 651, 1255, 8376, 1143, 5319, 2054, 10478]

## p=200, snr=5

In [16]:
howlong(snr=5, p=200)

5419

## p=200, snr=7, 8th step -- the first noise variable when selection works perfectly!

In [17]:
[howlong(snr=7, p=200, maxstep=8) for _ in range(10)]

[209, 126, 187, 0, 162, 645, 532, 292, 550, 393]

## p=200, snr=7, 10th step

In [ ]:
[howlong(snr=7, p=200, maxstep=10) for _ in range(10)]

[7, 10831, 17831, 7242, 2714, 1421, 12, 9645, 2625, 3093]

## p=200, snr=7, 15th step

In [ ]:
[howlong(snr=7, p=200, maxstep=15) for _ in range(10)]